# Dogs vs Cats Redux

In [1]:
%matplotlib inline

In [2]:
import utils; reload(utils)
from utils import *

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 960M (CNMeM is disabled, cuDNN 5110)
Using Theano backend.


In [3]:
path = 'data/data/redux/'

In [4]:
batch_size = 16

# Todo : 
1. create validation set and sample
2. move to separate dirs for each set
3. finetune and train
4. submit

# Create validation set and sample

In [ ]:
%cd ../data/redux

In [ ]:
%cd train

In [ ]:
%mkdir ../valid

In [ ]:
!pwd

In [ ]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(2000): os.rename(shuf[i], '../valid/' + shuf[i])

In [ ]:
%mkdir ../sample
%mkdir ../sample/train
%mkdir ../sample/valid

In [ ]:
from shutil import copyfile

In [ ]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(200): copyfile(shuf[i], '../sample/train/' + shuf[i])

In [ ]:
%cd ../valid

In [ ]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(50): copyfile(shuf[i], '../sample/valid/' + shuf[i])

# Move to separate dirs for each set

In [ ]:
%cd ../train

In [ ]:
%mkdir cats
%mkdir dogs

In [ ]:
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

In [ ]:
%cd ../valid

In [ ]:
%mkdir cats
%mkdir dogs

In [ ]:
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

In [ ]:
%cd ../sample/train/

In [ ]:
%mkdir cats
%mkdir dogs

In [ ]:
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

In [ ]:
%cd ../valid

In [ ]:
%mkdir cats
%mkdir dogs

In [ ]:
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

# Finetune and train 

In [5]:
from vgg16 import Vgg16

In [6]:
vgg = Vgg16()

/home/garvit/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:577: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [7]:
batches = vgg.get_batches(path + 'sample/train/', batch_size = batch_size)
val_batches = vgg.get_batches(path + 'sample/valid/', batch_size = batch_size)

Found 200 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


In [8]:
vgg.finetune(batches)

In [9]:
vgg.fit(batches, val_batches, nb_epoch = 1)

Epoch 1/1
200/200 [==============================] - 9s - loss: 1.1455 - acc: 0.7150 - val_loss: 0.3758 - val_acc: 0.8600


In [10]:
vgg.model.save_weights(path + 'results/ft1.h5')

In [11]:
vgg.model.load_weights(path + 'results/ft1.h5')

# Perform testing on single image

In [ ]:
from keras.preprocessing import image

In [ ]:
img = image.load_img('../data/redux/test/unknown/cat2.jpg')

In [ ]:
plt.imshow(img)

In [ ]:
from scipy.misc import imresize, imsave

img = imresize(img, (224, 224))
img = image.img_to_array(img)

img.shape

In [ ]:
img = img.reshape(1, 3, 224, 224)

In [ ]:
classes = ["cat", "dog"]
print("This is a {0}.".format(vgg.model.predict_classes(img)))

# Submit

In [ ]:
batches, preds = vgg.test(path + 'test/', batch_size = batch_size * 2)

In [ ]:
??vgg.test()

In [ ]:
filenames = batches.filenames

In [ ]:
preds[:5]

In [ ]:
filenames[:5]

In [ ]:
save_array(path+'results/test_preds.dat', preds)
save_array(path+'results/filenames.dat', filenames)

In [ ]:
preds = load_array(path + 'results/test_preds.dat')
filenames = load_array(path + 'results/filenames.dat')

In [ ]:
from PIL import Image
Image.open(path + 'test/' + filenames[0])

In [ ]:
isdog = preds[:, 1]

In [ ]:
isdog[:5]

In [ ]:
ids = [int(f[8:f.find('.')]) for f in filenames]
ids[:5]

In [ ]:
subm = np.stack([ids, isdog], axis = 1)
subm[:5]

In [ ]:
np.savetxt('data/redux/subm98.csv', subm, fmt='%d,%.5f', header='id,label', comments='')

In [ ]:
from IPython.display import FileLink
FileLink('data/redux/subm98.csv')

In [ ]:
isdog = np.clip(preds[:, 1], 0.05, 0.95)

In [ ]:
isdog[:]

# Run a few more epochs....

In [12]:
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
200/200 [==============================] - 9s - loss: 0.3708 - acc: 0.9050 - val_loss: 0.1741 - val_acc: 0.9400


In [13]:
vgg.model.save_weights(path+'results/ft2.h5')

In [14]:
vgg.model.optimizer.lr = 0.01

In [15]:
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
200/200 [==============================] - 9s - loss: 0.0902 - acc: 0.9500 - val_loss: 0.1233 - val_acc: 0.9600


In [16]:
vgg.model.save_weights(path+'results/ft2_1.h5')

In [17]:
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
200/200 [==============================] - 11s - loss: 0.0558 - acc: 0.9750 - val_loss: 0.1194 - val_acc: 0.9400


In [18]:
vgg.model.save_weights(path+'results/ft2_2.h5')

In [19]:
vgg.model.load_weights(path+'results/ft2_2.h5')

# Retraining last layer

In [20]:
vgg.model.pop()

In [21]:
for layer in vgg.model.layers: layer.trainable=False

In [22]:
vgg.model.add(Dense(2, activation='softmax'))

In [23]:
opt = RMSprop(lr=0.1)
vgg.model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
200/200 [==============================] - 9s - loss: 1.2479 - acc: 0.8850 - val_loss: 0.9671 - val_acc: 0.9400






# Retraining more than last layer

In [25]:
layers = vgg.model.layers
layers

In [26]:
# Get the index of the first dense layer...
first_dense_idx = [index for index,layer in enumerate(layers) if type(layer) is Dense][0]
# ...and set this and all subsequent layers to trainable
for layer in layers[first_dense_idx:]: layer.trainable=True

In [27]:
vgg.model.optimizer.lr = 0.01

In [28]:
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
200/200 [==============================] - 9s - loss: 1.4063 - acc: 0.9050 - val_loss: 0.7785 - val_acc: 0.9400


In [29]:
vgg.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________